# Análise de Sentimentos usando SentiWordNet

Nesse notebook, utilizaremos uma abordagem diferente para analisar sentimentos. Aqui utilizaremos um léxico de sentimentos que define, para cada palavra, um índice de positividade, negatividade e objetividade (neutralidade). Utilizaremos o SentiWordNet para tal tarefa.

In [10]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.wsd import lesk
from nltk.corpus import sentiwordnet as swn

Primeiro vamos carregar a base de teste dos reviews de filmes do IMDb.

In [11]:
reviews_test = []
for line in open('../Exercício 1/data/full_test.txt', 'r', encoding="utf8"):
    reviews_test.append(line.strip())

Criamos nossos rótulos para essa base de dados. Como vimos, para essa base, os primeiros 12500 reviews são positivos, o resto é negativo.

In [12]:
y_test = [1 if i < 12500 else 0 for i in range(25000)]

Vamos tratar a nossa base, removendo pontuação e tags HTML.

In [13]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_test_clean = preprocess_reviews(reviews_test)

Vamos usar o processo de tokenização do NLTK para separar os reviews por palavras.

In [14]:
tokens = []
for text in reviews_test_clean:
    words = nltk.word_tokenize(text)
    tokens.append(words)

In [ ]:
tokens[:5]

Como estamos analisando cada palavra, temos que remover as stopwords dos reviews. Stopwords geralmente possuem sentimento neutro (objetivo), assim, a remoção facilita na hora de analisar.

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
stop_words = set(stopwords.words('english'))
filter_tokens = []
for t in tokens:
    filter_tokens.append([w for w in t if not w in stop_words])

In [ ]:
filter_tokens[:5]

Vamos recuperar os índices de cada palavra, criando uma tupla contendo os índices positivos, negativos e de objetividade.

In [20]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\MM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\sentiwordnet.zip.


True

In [21]:
scores = []
for t in filter_tokens:
    synsets = [lesk(t, w) for w in t]
    sents = []
    synsets = list(filter(None, synsets))
    for s in synsets:
        senti_ss = swn.senti_synset(s.name())
        sents.append((senti_ss.pos_score(), senti_ss.neg_score(), senti_ss.obj_score()))
    scores.append(sents)

In [ ]:
scores[:5]

Aqui fazemos a classificação para cada review. Para isso, verificamos qual índice é maior e contamos o número de palavras com índice positivo maior e o número de palavras com índice negativo maior. Ignoramos as palavras com índice de objetividade maior por serem neutras. Após isso, calculamos a razão de índices positivos por negativos e a razão inversa (negativos por positivos). A maior razão indica a classe do review.

Crie um código que:

* Conte palavras positivas (índice positivo maior que negativo e objetivo)
* Conte palavras negativas (índice negativo maior que positivo e objetivo)
* Ignore palavras neutras (índice objetivo maior que positivo e negativo)
* Calcule a razão positiva (contagem positiva dividido pela contagem negativa)
* Calcule a razão negativa (contagem negativa dividido pela contagem positiva)
* Decida a classe: 1 (positivo - razão pos > razão neg), 0 (negativo - razão neg > razão pos)
* Adicionar a classe na lista classes

In [23]:
classes = []

for sc in scores:
    c_pos = 0
    c_neg = 0
    c_obj = 0
    for s in sc:
        if s[0] > s[1] and s[0] > s[2]:
            c_pos+=1
        if s[1] > s[0] and s[1] > s[2]:
            c_neg+=1
        if s[2] > s[0] and s[2] > s[1]:
            c_obj+=1
    
    #calculo da razão com smoothing (add-one)
    ratio_pos = (c_pos + 1) / (c_neg + 1)
    ratio_neg = (c_neg + 1) / (c_pos + 1)
    
    if ratio_pos > ratio_neg:
        classes.append(1)
    else:
        classes.append(0)

Quão preciso é nossa classificação? Vamos utilizar as classes reais dos reviews para calcular a acurácia.
Dica: a acurácia é em torno de 65%

In [24]:
from sklearn.metrics import accuracy_score

#Complete a linha abaixo
accuracy = accuracy_score(y_test, classes)

print ("Accuracy : %s" % (accuracy))

Accuracy : 0.65652
